In [94]:
from NLP_CS.src.clearml import safe_init_clearml
task=safe_init_clearml(project_name="NLP_CS",task_name="Refined Prompt")

UsageError: Current task already created and requested task name 'Refined Prompt/1' does not match current task name 'Refined Prompt'. If you wish to create additional tasks use `Task.create`, or close the current task with `task.close()` before calling `Task.init(...)`

In [ ]:
import os 
DATA_PATH="NLP_CS/data/"
train_data=os.path.join(DATA_PATH,"traindata.csv")
test_data=os.path.join(DATA_PATH,"devdata.csv")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [95]:
from ollama import chat
from ollama import ChatResponse



In [96]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
# load the csv  by hand
from NLP_CS.src.incontextclassif import * 

ds_train=load_data(train_data)
ds_test=load_data(test_data)
ds_test

Dataset({
    features: ['polarity', 'Aspect_Category', 'Target_term', 'Character_offset', 'labels', 'Sentence'],
    num_rows: 376
})

In [ ]:

ds_train = ds_train.map(remap_aspect)

Map: 100%|██████████| 1503/1503 [00:00<00:00, 16639.14 examples/s]


In [ ]:

# Select N examples from training set as demonstrations
demos = ds_train.select([0,1,3])  # example: use 3-shot

# Predict on a single test instance
for i in range(83,84):
    example = ds_test[i]
    pred = predict_sentiment(example, demos,build_prompt=build_prompt)
    #95 à voir
    print("Predicted sentiment:", pred)
    print("True label:", example['polarity'])
    normalize_prediction(pred,example)
example

Predicted sentiment: Positive
True label: positive


{'polarity': 'positive',
 'Aspect_Category': 'SERVICE#GENERAL',
 'Target_term': 'service',
 'Character_offset': '13:20',
 'labels': 0,
 'Sentence': 'Always great service!'}

In [100]:
len(ds_test)

376

In [ ]:

# Predict for all train examples
demonstrations=ds_test
predictions = []
for i in range(len(ds_test)):
    example = ds_test[i]
    print("considered example",example)
    prompt = build_prompt(example, demonstrations)
    
    try:
        response = chat(model='gemma3:1b', messages=[
            {"role": "user", "content": prompt}
        ])
        raw_pred = response['message']['content']
        print("raw pred",raw_pred)
        norm_pred = normalize_prediction(raw_pred,example)
    except Exception as e:
        print("the exception",e)
        norm_pred = "invalid"
    
    predictions.append(norm_pred)
    print(f"[{i+1}/{len(ds_test)}] Predicted: {norm_pred} | True: {example['polarity']}")

# Convert to DataFrame for analysis
results_df = pd.DataFrame({
    "true": [ds_test[i]["polarity"] for i in range(len(ds_test))],
    "predicted": predictions
})

# Accuracy calculation
valid_preds = results_df[results_df["predicted"] != "invalid"]
accuracy = (valid_preds["true"] == valid_preds["predicted"]).mean()
results_df.to_csv("results.csv", index=False)
print(f"\nIn-Context Learning Accuracy on Train Set (valid predictions only): {accuracy:.4f}")
print(f"Invalid predictions: {len(results_df) - len(valid_preds)} out of {len(results_df)}")

considered example {'polarity': 'positive', 'Aspect_Category': 'LOCATION#GENERAL', 'Target_term': 'neighborhood', 'Character_offset': '54:66', 'labels': 0, 'Sentence': 'great food, great wine list, great service in a great neighborhood...'}
raw pred positive
[1/376] Predicted: positive | True: positive
considered example {'polarity': 'negative', 'Aspect_Category': 'RESTAURANT#GENERAL', 'Target_term': 'place', 'Character_offset': '15:20', 'labels': 1, 'Sentence': 'I thought this place was totally overrated.'}
raw pred negative
[2/376] Predicted: negative | True: negative
considered example {'polarity': 'positive', 'Aspect_Category': 'FOOD#QUALITY', 'Target_term': 'Fish', 'Character_offset': '0:4', 'labels': 0, 'Sentence': 'Fish is so very fresh.'}
raw pred positive
[3/376] Predicted: positive | True: positive
considered example {'polarity': 'negative', 'Aspect_Category': 'SERVICE#GENERAL', 'Target_term': 'manager', 'Character_offset': '19:26', 'labels': 1, 'Sentence': 'I showed it to th

## Deepseek prompt

In [ ]:
def build_prompt(example, demonstrations):

    return prompt

In [ ]:

# Predict for all train examples
demonstrations=ds_test
predictions = []
for i in range(len(ds_test)):
    example = ds_test[i]
    print("considered example",example)
    prompt = build_prompt(example, demonstrations)
    
    try:
        response = chat(model='gemma3:1b', messages=[
            {"role": "user", "content": prompt}
        ])
        raw_pred = response['message']['content']
        print("raw pred",raw_pred)
        norm_pred = normalize_prediction(raw_pred,example)
    except Exception as e:
        print("the exception",e)
        norm_pred = "invalid"
    
    predictions.append(norm_pred)
    print(f"[{i+1}/{len(ds_test)}] Predicted: {norm_pred} | True: {example['polarity']}")

# Convert to DataFrame for analysis
results_df = pd.DataFrame({
    "true": [ds_test[i]["polarity"] for i in range(len(ds_test))],
    "predicted": predictions
})

# Accuracy calculation
valid_preds = results_df[results_df["predicted"] != "invalid"]
accuracy = (valid_preds["true"] == valid_preds["predicted"]).mean()
results_df.to_csv("results.csv", index=False)
print(f"\nIn-Context Learning Accuracy on Train Set (valid predictions only): {accuracy:.4f}")
print(f"Invalid predictions: {len(results_df) - len(valid_preds)} out of {len(results_df)}")